# Human Microbiome Project Data Analysis

This notebook performs various analyses on the Human Microbiome Project database, including genome size distribution, gene analysis, and taxonomic distribution.

In [ ]:
!pip install duckdb pandas plotly seaborn matplotlib numpy

In [ ]:
# Importing required libraries
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os 

# Connect to the database in parent directory\n",
db_path = os.path.join('..', 'hmp_microbiome.duckdb')
conn = duckdb.connect(db_path, read_only=True)


## 1. Genome Size Distribution Analysis

In [ ]:
def plot_genome_size_distribution():
    query = """
    SELECT organism_name, genome_size, 
           COUNT(g.gene_id) as gene_count
    FROM organisms o
    LEFT JOIN genes g ON o.organism_id = g.organism_id
    GROUP BY organism_name, genome_size
    ORDER BY genome_size DESC
    LIMIT 20
    """
    df = conn.execute(query).df()
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=('Genome Size vs Gene Count', 'Top 20 Organisms by Genome Size'))
    
    # Scatter plot
    fig.add_trace(
        go.Scatter(x=df['genome_size'], y=df['gene_count'], 
                  mode='markers', text=df['organism_name'],
                  name='Organisms'),
        row=1, col=1
    )
    
    # Bar plot
    fig.add_trace(
        go.Bar(x=df['organism_name'], y=df['genome_size'],
               name='Genome Size'),
        row=1, col=2
    )
    
    fig.update_layout(height=500, width=1200, showlegend=False)
    fig.update_xaxes(tickangle=45, row=1, col=2)
    return fig

# Generate and display the plot
fig1 = plot_genome_size_distribution()
fig1.show()

## 2. Gene Distribution Analysis

In [ ]:
def plot_gene_distributions():
    query = """
    SELECT o.organism_name,
           COUNT(CASE WHEN g.product LIKE '%hypothetical%' THEN 1 END) as hypothetical_genes,
           COUNT(CASE WHEN g.product NOT LIKE '%hypothetical%' THEN 1 END) as known_genes
    FROM organisms o
    LEFT JOIN genes g ON o.organism_id = g.organism_id
    GROUP BY o.organism_name
    ORDER BY (hypothetical_genes + known_genes) DESC
    LIMIT 15
    """
    df = conn.execute(query).df()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Known Genes',
        x=df['organism_name'],
        y=df['known_genes'],
        marker_color='rgb(55, 83, 109)'
    ))
    fig.add_trace(go.Bar(
        name='Hypothetical Genes',
        x=df['organism_name'],
        y=df['hypothetical_genes'],
        marker_color='rgb(26, 118, 255)'
    ))
    
    fig.update_layout(
        barmode='stack',
        title='Distribution of Known vs Hypothetical Genes Among Top 15 Organisms',
        xaxis_tickangle=45,
        height=500,
        width=1000
    )
    return fig

# Generate and display the plot
fig2 = plot_gene_distributions()
fig2.show()

## 3. Gene Product Analysis

In [ ]:
def plot_top_gene_products():
    query = """
    SELECT product, COUNT(*) as count
    FROM genes
    WHERE product NOT LIKE '%hypothetical%'
    GROUP BY product
    ORDER BY count DESC
    LIMIT 20
    """
    df = conn.execute(query).df()
    
    fig = px.treemap(df, 
                    path=[px.Constant("Gene Products"), 'product'],
                    values='count',
                    title='Top 20 Gene Products Distribution')
    fig.update_layout(height=600, width=1000)
    return fig

# Generate and display the plot
fig3 = plot_top_gene_products()
fig3.show()

## 4. Taxonomy Analysis

In [ ]:
def plot_taxonomy_distribution():
    query = """
    SELECT 
        SPLIT_PART(taxonomy, ';', 2) as phylum,
        COUNT(*) as organism_count,
        AVG(genome_size) as avg_genome_size
    FROM organisms
    GROUP BY phylum
    HAVING phylum != ''
    ORDER BY organism_count DESC
    """
    df = conn.execute(query).df()
    
    # Specify that the first subplot should be 'domain' type for pie chart
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'domain'}, {'type': 'xy'}]],
        subplot_titles=('Organism Count by Phylum', 'Average Genome Size by Phylum')
    )
    
    fig.add_trace(
        go.Pie(labels=df['phylum'],
               values=df['organism_count'],
               hole=0.3),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(x=df['phylum'],
               y=df['avg_genome_size'],
               marker_color='rgb(158,202,225)'),
        row=1, col=2
    )
    
    fig.update_layout(height=500, width=1200)
    fig.update_xaxes(tickangle=45, row=1, col=2)
    return fig

# Generate and display the plot
fig4 = plot_taxonomy_distribution()
fig4.show()

## 5. Gene Strand Distribution

In [ ]:
def plot_strand_distribution():
    query = """
    SELECT o.organism_name,
           COUNT(CASE WHEN g.strand = '+' THEN 1 END) as positive_strand,
           COUNT(CASE WHEN g.strand = '-' THEN 1 END) as negative_strand
    FROM organisms o
    JOIN genes g ON o.organism_id = g.organism_id
    GROUP BY o.organism_name
    ORDER BY (positive_strand + negative_strand) DESC
    LIMIT 10
    """
    df = conn.execute(query).df()
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
        name='Positive Strand',
        x=df['organism_name'],
        y=df['positive_strand'],
        marker_color='rgb(55, 83, 109)'
    ))
    fig.add_trace(go.Bar(
        name='Negative Strand',
        x=df['organism_name'],
        y=df['negative_strand'],
        marker_color='rgb(26, 118, 255)'
    ))
    
    fig.update_layout(
        barmode='group',
        title='Gene Strand Distribution in Top 10 Organisms',
        xaxis_tickangle=45,
        height=500,
        width=1000
    )
    return fig

# Generate and display the plot
fig5 = plot_strand_distribution()
fig5.show()

## 6. Gene Density Analysis

In [ ]:
def plot_gene_density():
    query = """
    SELECT o.organism_name,
           o.genome_size,
           COUNT(*) as gene_count,
           CAST(COUNT(*) AS FLOAT) / (o.genome_size / 1000000.0) as genes_per_mb
    FROM organisms o
    JOIN genes g ON o.organism_id = g.organism_id
    GROUP BY o.organism_name, o.genome_size
    ORDER BY genes_per_mb DESC
    LIMIT 15
    """
    df = conn.execute(query).df()
    
    fig = px.scatter(df,
                    x='genome_size',
                    y='genes_per_mb',
                    size='gene_count',
                    text='organism_name',
                    title='Gene Density Analysis')
    
    fig.update_traces(textposition='top center')
    fig.update_layout(height=600, width=1000)
    return fig

# Generate and display the plot
fig6 = plot_gene_density()
fig6.show()

In [ ]:
# Close the database connection
conn.close()